### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


def remove_punctuation(text):
    import string
    if isinstance(text, str):
        translator = str.maketrans('', '', string.punctuation)
        return text.translate(translator)
    return text
baby_df = pd.read_csv('./data/amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [243]:
#a)

baby_df["review"] = baby_df["review"].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

I had to modify the function a bit, because it was trying to remove punctuation from a float. Now it works as expected.

In [244]:
#b)
baby_df["review"] = baby_df["review"].fillna("")

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

Now there are no NaN values in the dataset.

In [245]:
#c)
baby_df = baby_df[baby_df['rating'] != 3].reset_index(drop=True)

#short test:
sum(baby_df["rating"] == 3)

0

There is no rating equal to 3 in the dataset.

In [246]:
#d) 
baby_df["rating"] = np.where(baby_df["rating"] >= 4, 1, -1)

#short test:
sum(baby_df["rating"]**2 != 1)

#163lk

0

Now we have only two classes of ratings. 1 or -1

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [247]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


I had to change the function get_feature_names() to get_feature_names_out() because of the deprecation. From certain version of sklearn, the function get_feature_names() has been replaced with get_feature_names_out().

In [248]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [249]:
#a)
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(baby_df["review"], baby_df["rating"], test_size=0.2, random_state=42)


In [ ]:
#b)

vectorizer = CountVectorizer()

train_x = vectorizer.fit_transform(train_x)
test_x = vectorizer.transform(test_x)


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [ ]:
#a)
model = LogisticRegression(max_iter = 1000)

model.fit(train_x, train_y)


In [ ]:
#b)
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

sorted_indices = np.argsort(coefficients)

most_negative = [feature_names[i] for i in sorted_indices[:10]]
most_positive = [feature_names[i] for i in sorted_indices[-10:]]

print("Most negative words: ", most_negative)
print("Most positive words: ", most_positive)


#hint: model.coef_, vectorizer.get_feature_names()

In [ ]:
import datetime

train_x, test_x, train_y, test_y = train_test_split(baby_df["review"], baby_df["rating"], test_size=0.2, random_state=42)

s_time = datetime.datetime.now()

vectorizer = CountVectorizer()

train_x = vectorizer.fit_transform(train_x)
test_x = vectorizer.transform(test_x)

model = LogisticRegression(max_iter = 1000)
model.fit(train_x, train_y)

feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

sorted_indices = np.argsort(coefficients)

most_negative = [feature_names[i] for i in sorted_indices[:10]]
most_positive = [feature_names[i] for i in sorted_indices[-10:]]

print("Most negative words: ", most_negative)
print("Most positive words: ", most_positive)


predicted_sentiments = model.predict(test_x)
predicted_sentiments_proba = model.predict_proba(test_x)

e_time = datetime.datetime.now()
unlimited_vocab_time = e_time - s_time


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [ ]:
#a)
predicted_sentiments = model.predict(test_x)

In [ ]:
#b)
predicted_sentiments_proba = model.predict_proba(test_x)
#hint: model.predict_proba()

In [ ]:
#c) 

df = pd.DataFrame({
    "review": baby_df["review"].iloc[test_y.index],
    "true_rating": test_y.values,
    "predicted_rating": predicted_sentiments,
    "predicted_rating_proba": predicted_sentiments_proba[:, 1]
})

most_positive = df.sort_values("predicted_rating_proba", ascending=False).head(5)
most_negative = df.sort_values("predicted_rating_proba").head(5)
accuracy_1 = np.mean(predicted_sentiments == test_y)

print(f"Most positive reviews: {most_positive}")
print(f"Most negative reviews: {most_negative}")
print(f"Accuracy: {accuracy_1}")

#hint: use the results of b)

In [ ]:
#d) 


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [ ]:
#a)
import datetime

train_x, test_x, train_y, test_y = train_test_split(baby_df["review"], baby_df["rating"], test_size=0.2, random_state=42)

s_time = datetime.datetime.now()

vectorizer = CountVectorizer(vocabulary=significant_words)

train_x = vectorizer.fit_transform(train_x)
test_x = vectorizer.transform(test_x)

model = LogisticRegression(max_iter = 1000)
model.fit(train_x, train_y)

feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

sorted_indices = np.argsort(coefficients)

most_negative = [feature_names[i] for i in sorted_indices[:10]]
most_positive = [feature_names[i] for i in sorted_indices[-10:]]

print("Most negative words: ", most_negative)
print("Most positive words: ", most_positive)


predicted_sentiments = model.predict(test_x)
predicted_sentiments_proba = model.predict_proba(test_x)

e_time = datetime.datetime.now()
limited_vocab_time = e_time - s_time

df = pd.DataFrame({
    "review": baby_df["review"].iloc[test_y.index],
    "true_rating": test_y.values,
    "predicted_rating": predicted_sentiments,
    "predicted_rating_proba": predicted_sentiments_proba[:, 1]
})


most_positive = df.sort_values("predicted_rating_proba", ascending=False).head(5)
most_negative = df.sort_values("predicted_rating_proba").head(5)

accuracy_2 = np.mean(predicted_sentiments == test_y)

print(f"Most positive reviews: {most_positive}")
print(f"Most negative reviews: {most_negative}")
print(f"Accuracy: {accuracy_2}")


In [ ]:
#b)

feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

word_impact = pd.DataFrame({
    "word": feature_names,
    "impact": coefficients
}).sort_values("impact", ascending=False)

print("\nWord_impact\n", word_impact)


In [ ]:
#c)
from datetime import time

print(f"Time of evaluation without limited vocabulary (1): {unlimited_vocab_time}")
print(f"Time of evaluation with limited vocabulary (2): {limited_vocab_time}")
print(f"Time of evaluation difference (1-2): {abs(unlimited_vocab_time - limited_vocab_time)}")

print(f"Accuracy without limited vocabulary (1): {accuracy_1}")
print(f"Accuracy with limited vocabulary (2): {accuracy_2}")
print(f"Accuracy difference (2): {abs(accuracy_2 - accuracy_1)}")

#hint: %time, %timeit